XGBoost
===

In [215]:
import xgboost as xgb
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
import scipy as sci
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV

Algorithms
---

In [216]:
# Parsing
train_set = pd.read_csv('dataset/output/train_local_dataset_polimi_extended_no2014.csv',delimiter=',')
test_set = pd.read_csv('dataset/output/test_local_dataset_polimi_extended_no2014.csv',delimiter=',')
pd.options.mode.chained_assignment = None

train_set = train_set[['area','sottoarea','timestamp','categoria','vendite']]
test_set = test_set[['area','sottoarea','timestamp','categoria']]

In [217]:
train_set.head()

,zona,area,sottoarea,categoria,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,settimana_dell_anno,mese,...,regione_PUG,regione_SAR,regione_SIC,regione_TAA,regione_TOS,regione_UMB,regione_VEN,zona_italia_Centro,zona_italia_Nord,zona_italia_Sud
0,1,1,5,1,1420070400,3,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,5,1,1420156800,4,2,2,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,1,5,1,1420243200,5,3,3,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,1,5,1,1420329600,6,4,4,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,1,5,1,1420416000,0,5,5,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [218]:
test_set = test_set.drop('vendite',1)
test_set.head()

,zona,area,sottoarea,categoria,timestamp,giorno_della_settimana,giorno_del_mese,giorno_dell_anno,settimana_dell_anno,mese,...,regione_PUG,regione_SAR,regione_SIC,regione_TAA,regione_TOS,regione_UMB,regione_VEN,zona_italia_Centro,zona_italia_Nord,zona_italia_Sud
0,1,1,5,1,1462838400,1,10,131,19,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,5,1,1462924800,2,11,132,19,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,1,5,1,1463011200,3,12,133,19,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,1,5,1,1463097600,4,13,134,19,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,1,5,1,1463184000,5,14,135,19,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [219]:
y_train = train_set.pop('vendite')

In [40]:
cv_params = { 'max_depth':[1,10,100],'min_child_weight':[1,10,100] }
ind_params = { 'subsample':0.9,'learning_rate':0.01,'n_estimators': 500, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', "nthread":8}

optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params),
                            cv_params,
                            scoring = 'accuracy',
                            cv = 3, 
                            n_jobs = -1)

In [ ]:
optimized_GBM.fit(train_set,y_train)

In [12]:
optimized_GBM.grid_scores_

[mean: 0.25241, std: 0.01767, params: {'max_depth': 10, 'min_child_weight': 10},
 mean: 0.25140, std: 0.01398, params: {'max_depth': 10, 'min_child_weight': 70},
 mean: 0.25334, std: 0.01664, params: {'max_depth': 70, 'min_child_weight': 10},
 mean: 0.25571, std: 0.01535, params: {'max_depth': 70, 'min_child_weight': 70}]

In [220]:
xgdmat = xgb.DMatrix(train_set, y_train, missing=np.nan)

In [14]:
our_params = {'eta': 0.1, 'seed':0,'learning_rate':0.01, 'subsample': 0.9, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':70, 'min_child_weight':70, "nthread":8} 
# Grid Search CV optimized settings

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'], 
                early_stopping_rounds = 100)

Will train until cv error hasn't decreased in 100 rounds.
Stopping. Best iteration: 562


In [221]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':10, 'min_child_weight':10} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 562)

In [222]:
testdmat = xgb.DMatrix(test_set, missing=np.nan)

In [223]:
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred

array([ 9.13438892,  9.00417519,  8.97003746, ...,  2.34735298,
        2.166327  ,  2.91668844], dtype=float32)

In [334]:
final_df = test_set.copy()

In [335]:
final_df['prediction'] = y_pred

In [286]:
final_df['prediction'] = final_df['prediction'].map(lambda x: round(x))
    
print final_df.head()['prediction']

0    9.0
1    9.0
2    9.0
3    9.0
4    2.0
Name: prediction, dtype: float64


ENSEMBLE
===
Dati ARMA
---

In [316]:
p1 = pd.read_csv('/home/ubuntu/forecasts.csv',delimiter=',')
print p1.head()

   Sottoarea         p1        p2        p3        p4        p5        p6  \
0          1   3.696093  2.845264  2.298765  2.211461  2.362665  2.516626   
1          2   6.453488  4.209412  3.562813  4.655359  4.274754  4.155359   
2          3   1.749039  0.731649  1.055015  1.855050  1.179213  0.746936   
3          4   5.947260  5.235588  4.950944  4.525222  4.555232  4.669086   
4          5  10.604680  9.872195  7.424551  5.611206  5.254126  6.115085   

         p7        p8        p9       p10  
0  2.576818  2.562199  2.526496  2.504315  
1  4.587996  4.155652  4.395151  4.391570  
2  1.580534  1.595378  0.809224  1.162431  
3  4.573520  4.609151  4.612898  4.598645  
4  7.092021  7.582609  7.394454  6.965099  


In [317]:
p2 = pd.read_csv('/home/ubuntu/forecasts2.csv',delimiter=',')
print p1.head()

   Sottoarea         p1        p2        p3        p4        p5        p6  \
0          1   3.696093  2.845264  2.298765  2.211461  2.362665  2.516626   
1          2   6.453488  4.209412  3.562813  4.655359  4.274754  4.155359   
2          3   1.749039  0.731649  1.055015  1.855050  1.179213  0.746936   
3          4   5.947260  5.235588  4.950944  4.525222  4.555232  4.669086   
4          5  10.604680  9.872195  7.424551  5.611206  5.254126  6.115085   

         p7        p8        p9       p10  
0  2.576818  2.562199  2.526496  2.504315  
1  4.587996  4.155652  4.395151  4.391570  
2  1.580534  1.595378  0.809224  1.162431  
3  4.573520  4.609151  4.612898  4.598645  
4  7.092021  7.582609  7.394454  6.965099  


In [336]:
test_set['vendite'] = y_pred
final_df = test_set.copy()[['sottoarea','categoria','timestamp','vendite']]
final_df.head()

,sottoarea,categoria,timestamp,vendite
0,5,1,1462838400,9.134389
1,5,1,1462924800,9.004175
2,5,1,1463011200,8.970037
3,5,1,1463097600,9.163619
4,5,1,1463184000,2.202881


In [337]:
def eval_mean(category, subarea, iterator, value):
    if category == 1.:
        new_value = p1[(p1['Sottoarea'] == subarea)]['p'+str(iterator)].item()
        if new_value != None:
            return value*0.7 + new_value*0.3
        else:
            return value
    elif category == 2.:
        new_value = p2[(p2['Sottoarea'] == subarea)]['p'+str(iterator)].item()
        if new_value != None:
                return value*0.7 + new_value*0.3
        else:
            return value

In [ ]:
for cat in range(1,3):
    for i in range(1,146):
        if (i in final_df.sottoarea.values) and (i in p1.Sottoarea.values) and (i in p2.Sottoarea.values) :
            iterator = 1
            for row in final_df[(final_df['categoria'] == cat) & (final_df['sottoarea'] == i)].iterrows():
                final_df.loc[(final_df['categoria'] == cat)\
                             & (final_df['sottoarea'] == i)\
                             & (final_df['timestamp'] == row[1]['timestamp']),\
                             'vendite'] = eval_mean(cat, i, iterator, row[1]['vendite'])
                iterator += 1

In [338]:
final_df['vendite'] = final_df['vendite'].map(lambda x: round(x))

In [339]:
# MAPE Eval

dddd = pd.read_csv("dataset/output/test_local_dataset_polimi_extended_no2014.csv",delimiter=",")
sub_area_list = list(dddd[['sottoarea','categoria','vendite']].values)
sub_area_mean = {}
for i in sub_area_list:
    if sub_area_mean.get((i[0],i[1]))!=None:
        sub_area_mean[(i[0],i[1])] += [i[2]]
    else:
        sub_area_mean[(i[0],i[1])] = [i[2]]
for i in sub_area_mean:
    sub_area_mean[i] = np.mean(sub_area_mean[i]) if np.mean(sub_area_mean[i])>0. else 3.4534722
len(sub_area_mean)

gt = pd.read_csv('dataset/output/test_local_dataset_polimi_extended_no2014.csv',delimiter=',')




gt_dict={}
for i in range(len(gt['vendite'])):
    a = gt_dict.get((gt['sottoarea'][i],gt['categoria'][i]))
    if a == None:
        gt_dict[(gt['sottoarea'][i],gt['categoria'][i])] = [gt['vendite'][i]]
    else:
        gt_dict[(gt['sottoarea'][i],gt['categoria'][i])] += [gt['vendite'][i]]

fin_dict={}
for i in range(len(final_df['vendite'])):
    a = fin_dict.get((final_df['sottoarea'][i],final_df['categoria'][i]))
    if a == None:
        fin_dict[(final_df['sottoarea'][i],final_df['categoria'][i])] = [final_df['vendite'][i]]
    else:
        fin_dict[(final_df['sottoarea'][i],final_df['categoria'][i])] += [final_df['vendite'][i]]

mape_error = 0 
for i in gt_dict:
    attual_rate_list= gt_dict[i]
    predict_list = fin_dict[i]
    #denominator = attual_rate if attual_rate !=0 else sub_area_mean[(gt['sottoarea'][i],gt['categoria'][i])]
    error = sum([np.abs((attual_rate_list[i]-predict_list[i])/(attual_rate_list[i] if attual_rate_list[i] !=0 else sub_area_mean[(gt['sottoarea'][i],gt['categoria'][i])]))
                 for i in range(len(attual_rate_list))])/len(attual_rate_list)

    mape_error += error
print " original mape ", mape_error/len(gt_dict)

MAPE_error = 0
for i in gt_dict:
    attual_rate_list= gt_dict[i]
    predict_list = fin_dict[i]
    #denominator = attual_rate if attual_rate !=0 else sub_area_mean[(gt['sottoarea'][i],gt['categoria'][i])]
    error = max(np.abs(np.array(attual_rate_list)-np.array(predict_list)))

    MAPE_error += error

print "max absolut error",  MAPE_error/len(gt_dict)


 original mape  0.280503491286
max absolut error 2.36805555556
